In [1]:
# Import brmspy - works for both installed package and cloned repo
import sys
import os

try:
    # Try importing as installed package
    import brmspy
    from brmspy import brms
    print(f"✓ Using installed brmspy {brmspy.__version__}")
except ImportError:
    # Fall back to importing from cloned repo
    # Add parent directory to path (assumes running from examples/)
    repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    if repo_root not in sys.path:
        sys.path.insert(0, repo_root)
    
    try:
        import brmspy
        from brmspy import brms
        print(f"✓ Using brmspy from repository: {repo_root}")
        print(f"  Version: {brmspy.__version__}")
    except ImportError as e:
        print("✗ Could not import brmspy!")
        print("  Please either:")
        print("  1. Install: pip install brmspy")
        print("  2. Or run from repository root with: jupyter notebook examples/getting_started.ipynb")
        raise

# Import other required packages
import pandas as pd
import numpy as np

import arviz as az
import matplotlib.pyplot as plt

print("\n" + "="*60)
print("Setup complete!")
print("="*60)

Error importing in API mode: ImportError("dlopen(/Users/sebastian/PycharmProjects/pybrms/.venv/lib/python3.12/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <05EE76C0-33BF-3D4C-BC56-84005241BC88> /Users/sebastian/PycharmProjects/pybrms/.venv/lib/python3.12/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.


✓ Using brmspy from repository: /Users/sebastian/PycharmProjects/pybrms
  Version: 0.1.0

Setup complete!


In [2]:
df = brms.get_brms_data("epilepsy")

In [3]:
model = brms.fit(
    formula="count ~ zAge + zBase",
    data=df,
    family="poisson",
    priors=[
        ("normal(0, 0.5)", "b"),
        ("normal(1, 0.5)", "Intercept")
    ],
    algorithm="fullrank",
    iter=20000,        # max gradient steps for VI
    grad_samples=10,   # MC samples for gradient estimate
    elbo_samples=250,  # MC samples for ELBO estimate
    eval_elbo=100,     # recompute ELBO every 100 iters
    tol_rel_obj=1e-3,  # stop when relative ELBO change < 0.1%
    eta=1.0,           # step size scaling (keep default unless unstable)
    adapt_engaged=True
)


R callback write-console: Compiling Stan program...
  


Fitting model with brms (backend: cmdstanr)...


R callback write-console: Start sampling
  


------------------------------------------------------------ 
EXPERIMENTAL ALGORITHM: 
  This procedure has not been thoroughly tested and may be unstable 
  or buggy. The interface is subject to change. 
------------------------------------------------------------ 
Gradient evaluation took 2.3e-05 seconds 
1000 transitions using 10 leapfrog steps per transition would take 0.23 seconds. 
Adjust your expectations accordingly! 
Begin eta adaptation. 
Iteration:   1 / 250 [  0%]  (Adaptation) 
Iteration:  50 / 250 [ 20%]  (Adaptation) 
Iteration: 100 / 250 [ 40%]  (Adaptation) 
Iteration: 150 / 250 [ 60%]  (Adaptation) 
Iteration: 200 / 250 [ 80%]  (Adaptation) 
Success! Found best value [eta = 1] earlier than expected. 
Begin stochastic gradient ascent. 
  iter             ELBO   delta_ELBO_mean   delta_ELBO_med   notes  
   100       -12252.762             1.000            1.000 
   200         -919.201             6.665           12.330 
   300         -883.641             4.457       

In [5]:
model.idata

Inference data with groups:
	> posterior
	> observed_data

In [4]:
# This calculates Pareto k values
loo_result = az.loo(model.idata, pointwise=True)

print(loo_result)

# Check the k-values specifically
k_values = loo_result.pareto_k
print(f"Bad k values (>0.7): {sum(k_values > 0.7)}")

az.plot_khat(loo_result)
plt.show()

TypeError: log likelihood not found in inference data object

In [ ]:
brms._convert_R_to_python(model.brmsfit)

TypeError: _convert_R_to_python() missing 2 required positional arguments: 'data' and 'family'